In [ ]:
!pip install mysql-connector-python
import mysql.connector

     |████████████████████████████████| 38.0 MB 1.3 MB/s 


In [ ]:
mydb = mysql.connector.connect(
  host="host",
  user="user",
  password="pass"
)

import json
import pandas as pd
import numpy as np
mycursor = mydb.cursor()

def exec_q(qT):
  mycursor.execute(qT)
  myresult = mycursor.fetchall()
  jD = []
  for x in myresult:
    jD.append(x[0])
  JReturn = '['+','.join(jD)+']'
  return JReturn

def exec_sql(sql):
  mycursor.execute(sql)
  myresult = mycursor.fetchall()
  return myresult

def ex_msr_tipe(t):
  return exec_sql('select * from primaprod.master_sumber_referensi where tipe_ref = "'+t+'"')


In [ ]:
from datetime import datetime

In [ ]:
def exc_rsp(tF,tT,kP = ''):
  cond = ' 1=1 ' if kP == '' else ' kode_poli = "'+kP+'" '
  rw = exec_sql('SELECT sum(harga+uang_tusla) H\
  , group_concat(keterangan_json separator "++") K\
  , `apotek_resep_pasien`.`no_resep`, \
  concat("[",apotek_resep_pasien.no_rm,"] ",nama_pelanggan) pelanggan\
  ,tgl_resep_created\
  , group_concat(DISTINCT apotek_resep_pasien_detail.user separator "++") U, id_ruang, concat(instalasi_id, ".", id_smf) pl, \
  date_format(tgl_resep_created, "%y%m%d") T, apotek_resep_pasien.id as id, apotek_resep_pasien.* \
FROM primaprod.apotek_resep_pasien \
LEFT JOIN primaprod.`mastercr_no_kamar` ON `mastercr_no_kamar`.`id`=`id_no_kamar`  \
LEFT JOIN primaprod.`poli_kunjungan_pasien` ON `poli_kunjungan_pasien`.`id` = `detail_reg`   \
INNER JOIN primaprod.`apotek_resep_pasien_detail` ON `apotek_resep_pasien`.`no_resep` = `apotek_resep_pasien_detail`.`no_resep`    \
WHERE tgl_resep_created between "'+tF+' 00:00:00" AND "'+tT+' 23:59:59" AND apotek_resep_pasien.no_resep not like "%RE%"   \
AND '+cond+' \
GROUP BY `apotek_resep_pasien`.`no_resep`   \
ORDER BY `apotek_resep_pasien`.`tgl_resep_created`')
  lER = []
  for x in rw:
    js = dict()
    arr = x[1].split("++")
    js = json.loads(arr[0])
    nr = []
    tx = ''
    if 'eresep_ref' in js:
      tx = js['eresep_ref']['no_eresep']
    else:
      tx = "tanpa eresep"
    nr = [tx,x[2],x[3],x[4].strftime("%H:%M")] 
    lER.append(nr)
    #lER.append(x[1])
  #np.array(lER)
  return lER #np.unique(lER)

def exc_ersp(lER):
  rw = exec_sql('select kp,count(kp) from (SELECT if(kode_poli="300",concat("k",id_kamar),kode_poli) kp \
FROM primaprod.eprescription_resep_pasien \
WHERE no_resep in ("'+'","'.join(lER)+'")) a group by kp')
  return rw



In [ ]:
cb = exc_rsp('2022-01-11','2022-01-17','150')
cb = pd.DataFrame(cb, columns = ['nomor eresep ','nomor resep','identitas pemilik resep','waktu'])
cb

,nomor eresep,nomor resep,identitas pemilik resep,waktu
0,ERJ2201110028,RRJ2201110018,[2568163] SUSANTI,08:21
1,ERJ2201110030,RRJ2201110019,[1268984] BONARD DAVID NAPITUPULU,08:23
2,ERJ2201110034,RRJ2201110020,"[2570290] YULITA CHARYSMAWATI, NY",08:24
3,ERJ2201110037,RRJ2201110021,[2573345] CHAIRUNNISA RAZAK,08:25
4,ERJ2201110039,RRJ2201110022,[2567489] MARKONAH,08:28
...,...,...,...,...
102,tanpa eresep,RRJ2201170118,[2569175] SUMARTI,10:27
103,tanpa eresep,RRJ2201170124,[2567047] UNI NUR AINI,10:32
104,ERJ2201170368,RRJ2201170135,[2289234] JAMAN MANALU,10:46
105,tanpa eresep,RRJ2201170143,[2289234] JAMAN MANALU,10:51
